In [1]:
import pandas as pd
from datasets import load_dataset

In [2]:
metadata_raw = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Video_Games", trust_remote_code=True)

In [3]:
metadata_raw_df = metadata_raw['full'].to_pandas()

In [4]:
metadata_raw_df

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Video Games,Dash 8-300 Professional Add-On,5.0,1,[Features Dash 8-300 and 8-Q300 ('Q' rollout l...,[The Dash 8-300 Professional Add-On lets you p...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Aerosoft,"[Video Games, PC, Games]","{""Pricing"": ""The strikethrough price is the Li...",B000FH0MHO,None,None,None
1,Video Games,Phantasmagoria: A Puzzle of Flesh,4.1,18,[Windows 95],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Sierra,"[Video Games, PC, Games]","{""Best Sellers Rank"": {""Video Games"": 137612, ...",B00069EVOG,None,None,None
2,Video Games,NBA 2K17 - Early Tip Off Edition - PlayStation 4,4.3,223,[The #1 rated NBA video game simulation series...,[Following the record-breaking launch of NBA 2...,58.0,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['NBA 2K17 - Kobe: Haters vs Players...,2K,"[Video Games, PlayStation 4, Games]","{""Release date"": ""September 16, 2016"", ""Best S...",B00Z9TLVK0,None,None,None
3,Video Games,Nintendo Selects: The Legend of Zelda Ocarina ...,4.9,22,[Authentic Nintendo Selects: The Legend of Zel...,[],37.42,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Amazon Renewed,"[Video Games, Legacy Systems, Nintendo Systems...","{""Best Sellers Rank"": {""Video Games"": 51019, ""...",B07SZJZV88,None,None,None
4,Video Games,Thrustmaster Elite Fitness Pack for Nintendo Wii,3.0,3,"[Includes (9) Total Accessories, Pedometer, Wi...",[The Thrustmaster Motion Plus Elite Fitness Pa...,None,"{'hi_res': [None, None, None, None, None, None...","{'title': [], 'url': [], 'user_id': []}",THRUSTMASTER,"[Video Games, Legacy Systems, Nintendo Systems...","{""Release date"": ""November 1, 2009"", ""Pricing""...",B002WH4ZJG,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137264,None,Story of Seasons: Pioneers Of Olive Town (Nint...,4.5,397,[A wild world of discovery - tame the wilderne...,"[Product Description, Inspired by Tales of you...",31.04,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Marvelous Europe,"[Video Games, Nintendo Switch, Games]","{""Release date"": ""March 26, 2021"", ""Best Selle...",B09XQJS4CZ,None,None,None
137265,Video Games,MotoGP 18 (PC DVD) UK IMPORT REGION FREE,4.0,1,[Brand new game engine - MotoGP18 has been reb...,[Become the champion of the 2018 MotoGP Season...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Milestone,"[Video Games, Game Genre of the Month]","{""Pricing"": ""The strikethrough price is the Li...",B07DGPTGNV,None,None,None
137266,Cell Phones & Accessories,Century Accessory Soft Silicone Protective Ski...,2.9,19,"[Easy access to all buttons, controls and port...",[This soft case cover will add a splash of col...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Century Accessory,"[Video Games, Legacy Systems, Xbox Systems, Xb...","{""Package Dimensions"": ""2.76 x 2.76 x 0.2 inch...",B00HUWCQBW,None,None,None
137267,None,Hasbro Interactive Mr. Potato Head Activity Pa...,3.9,5,[],"[Amazon.com, Everyone's favorite master-of-dis...",None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Hasbro,"[Video Games, PC, Games]","{""Release date"": ""July 24, 1999"", ""Best Seller...",B00002S9MH,None,None,None


In [5]:
with pd.option_context('display.max_colwidth', None):
    display(metadata_raw_df.sample(10))

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
100343,Video Games,LEGO Ninjago: Shadow of Ronin - Nintendo 3DS,4.5,402,"[Battle through iconic locales from the LEGO Ninjago: Masters of Spinjitsu TV series, including the Ice Temple, the Toxic Bogs and Chen's Island., Uncover the secret behind the Obsidian Weapons, ancient yet powerful tools., Defend Ninjago from well-known enemies, including the Serpentine, Nindroids and more., Drive, fly, stomp and slide your way through levels with an exciting array of bikes, jets, mechs and even dragons!, Unleash an elemental tornado to smash your way through enemies and solve puzzles. Team up and create the multi-elemental Tornado of Creation!]","[There is a new threat in Ninjago, and he goes by the name of Ronin. With help from his army of dark samurai, Ronin steals the Ninjas' memories using an ancient weapon called the Obsidian Glaive. In LEGO Ninjago: Shadow of Ronin, it is up to players to help the Ninjas regain their memories and reclaim their powers before Ronin completes his plan and releases an even greater evil on Ninjago.]",8.49,"{'hi_res': ['https://m.media-amazon.com/images/I/914gqqAXReL._AC_SL1500_.jpg', 'https://m.media-amazon.com/images/I/51WzbguWjjL._AC_SL1500_.jpg'], 'large': ['https://m.media-amazon.com/images/I/614EU+osB9L._AC_.jpg', 'https://m.media-amazon.com/images/I/31BDm2VqflL._AC_.jpg'], 'thumb': ['https://m.media-amazon.com/images/I/614EU+osB9L._AC_US75_.jpg', 'https://m.media-amazon.com/images/I/31BDm2VqflL._AC_US75_.jpg'], 'variant': ['MAIN', 'GLMR']}","{'title': [], 'url': [], 'user_id': []}",WARNER BROS,"[Video Games, Legacy Systems, Nintendo Systems, Nintendo 3DS & 2DS, Games]","{""Manufacturer"": ""Warner Home Video - Games"", ""Part Number"": ""1000550191"", ""Item Weight"": ""1.6 ounces"", ""Product Dimensions"": ""0.51 x 5.29 x 4.94 inches"", ""Item model number"": ""1000550191"", ""Is Discontinued By Manufacturer"": ""No"", ""Color"": ""Original Version"", ""Item Package Quantity"": ""1"", ""Batteries Included?"": ""No"", ""Batteries Required?"": ""No"", ""Best Sellers Rank"": {""Video Games"": 18986, ""Nintendo 3DS & 2DS Games"": 140}, ""Date First Available"": ""January 10, 2015"", ""Brand"": ""WB Games"", ""Operating System"": ""Nintendo_3ds"", ""Hardware Platform"": ""Nintendo 3DS"", ""Genre"": ""Action-game-genre"", ""Publication Date"": ""March 24, 2015""}",B07MQ6DTLG,None,None,None
105534,Toys & Games,The Sims 3: Supernatural (Mac/PC DVD),4.4,329,[],"[The Sims 3: Supernatural is the seventh expansion pack for The Sims 3 on OS X and PC. It was released on September 4, 2012 in North America and Greece.]",None,"{'hi_res': ['https://m.media-amazon.com/images/I/918VAtzj3KL._AC_SL1500_.jpg', 'https://m.media-amazon.com/images/I/91FYWSxkdHL._AC_SL1500_.jpg'], 'large': ['https://m.media-amazon.com/images/I/51-0AyGpjNL._AC_.jpg', 'https://m.media-amazon.com/images/I/51iMGYbdtfL._AC_.jpg'], 'thumb': ['https://m.media-amazon.com/images/I/51-0AyGpjNL._AC_US40_.jpg', 'https://m.media-amazon.com/images/I/51iMGYbdtfL._AC_US40_.jpg'], 'variant': ['MAIN', 'BACK']}","{'title': [], 'url': [], 'user_id': []}",Electronic Arts,"[Video Games, PC, Games]","{""Product Dimensions"": ""7.48 x 5.31 x 0.59 inches"", ""Item Weight"": ""2.96 ounces"", ""Item model number"": ""PCOESIELE10905"", ""Best Sellers Rank"": {""Video Games"": 149372, ""Mac-compatible Games"": 649, ""PC-compatible Games"": 8251, ""Legacy Systems"": 56879}, ""Is Discontinued By Manufacturer"": ""No"", ""Release date"": ""November 23, 2012"", ""Manufacturer"": ""Electronic Arts"", ""Brand"": ""Electronic Arts"", ""Operating System"": ""Windows 7/XP/Vista"", ""Hardware Platform"": ""pc, mac"", ""Format"": ""DVD-ROM"", ""Genre"": ""life-simulation-games""}",B009RNWSOC,None,None,None
83735,Computers,"Pink Mechanical Gaming Keyboard and Mouse Combo Blue Switch 104 Keys White Backlit Keyboards, 7 But

# Build Feature Transformations

In [6]:
train_df = pd.read_parquet("../data/train.parquet")
val_df = pd.read_parquet("../data/val.parquet")

In [7]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline

In [8]:
# Custom function to reshape data from 1D to 2D
def reshape_2d_to_1d(X):
    return np.array(X).reshape(-1)

def flatten_string_array_col(X):
    return np.array(['\n'.join(x) for x in X])

def tfidf_pipeline_steps():
    steps = [
        ('impute', SimpleImputer(strategy='constant', fill_value='')),
        ('reshape', FunctionTransformer(reshape_2d_to_1d, validate=False)),
        ('tfidf', TfidfVectorizer(min_df=5, max_features=1000, ngram_range=(1, 2)))
    ]
    return steps

def description_pipeline_steps():
    steps = [
        ('flatten_string_array_col', FunctionTransformer(flatten_string_array_col, validate=False)),
        ('tfidf', TfidfVectorizer(min_df=5, max_features=1000, ngram_range=(1, 2)))
    ]
    return steps

def categories_pipeline_steps():
    def tokenizer(s):
        return s.split('\n')
    steps = [
        ('flatten_string_array_col', FunctionTransformer(flatten_string_array_col, validate=False)),
        ('tfidf', CountVectorizer(tokenizer=tokenizer, token_pattern=None))
    ]
    return steps

data = pd.Series(["from 14.99", "14.99", "price: 9.99", "20 dollars", "none"])
price_pattern = r'\b((?:\d+\.\d*)|(?:\d+))\b'
display(data.str.extract(price_pattern))

def price_parse_dtype(series, pattern):
    return series.str.extract(pattern).astype(float)

def price_pipeline_steps(price_pattern):
    steps = [
        ('extract_price', FunctionTransformer(price_parse_dtype, kw_args=dict(pattern=price_pattern), validate=False)),
        ('impute', SimpleImputer(strategy='constant', fill_value=0)),
        ('min_max_scale', MinMaxScaler())
    ]
    return steps

,0
0,14.99
1,14.99
2,9.99
3,20
4,NaN


In [9]:
tfm = [
    ('main_category', OneHotEncoder(handle_unknown='ignore'), ['main_category']),  # One-hot encoding for categorical field
    ('title', Pipeline(tfidf_pipeline_steps()), ['title']),  # TF-IDF vectorizer for text field
    ('description', Pipeline(description_pipeline_steps()), 'description'),  # TF-IDF vectorizer for another text field
    ('categories', Pipeline(categories_pipeline_steps()), 'categories'),  # Count Vectorizer for multi-label categorical
    ('price', Pipeline(price_pipeline_steps(price_pattern)), 'price')  # Normalizing price
]
cols = [x[0] for x in tfm]
cols

['main_category', 'title', 'description', 'categories', 'price']

In [10]:
train_full_df = pd.merge(train_df, metadata_raw_df[['parent_asin'] + cols], how='left', on='parent_asin')
# Drop duplicated item so that the Pipeline only fit the unique item features
fit_df = train_full_df.drop_duplicates(subset=['parent_asin'])
val_full_df = pd.merge(val_df, metadata_raw_df[['parent_asin'] + cols], how='left', on='parent_asin')
train_full_df

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price
0,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B0863MT183,4.0,1613701986538,All Electronics,KIWI design VR Stand Compatible with Quest 2/R...,[],"[Video Games, PC, Virtual Reality, Headsets]",19.99
1,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B08P8P7686,5.0,1613702112995,All Electronics,Seltureone Accessories Compatible for Quest 2 ...,[],"[Video Games, PC, Virtual Reality, Headsets]",20.99
2,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B0B7LV3DN2,4.0,1617641445475,Cell Phones & Accessories,Eyglo Replacement Elite Strap for Oculus/Meta ...,[],"[Video Games, PC, Virtual Reality, Headsets]",28.49
3,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B09WMQ6DXG,5.0,1620231368468,All Electronics,"AMVR Controller Grips for Quest 2, Controller ...",[],"[Video Games, PC, Virtual Reality, Controllers]",15.99
4,AFFZVSTUS3U2ZD22A2NPZSKOCPGQ,B01GW3LRD2,5.0,1491589434000,Video Games,Forza Horizon 3 – Xbox One,[You’re in charge of the Horizon Festival. Cus...,"[Video Games, Xbox One, Games]",22.0
...,...,...,...,...,...,...,...,...,...
54965,AF4VJ4NQ7LO256VSOVNQ6Q5PGNBA,B003LJSJXW,4.0,1293327789000,Video Games,Monopoly Streets - Playstation 3,"[Product Description, To celebrate its 75th an...","[Video Games, Legacy Systems, PlayStation Syst...",27.99
54966,AGRXRIPAZTGAQHFKXZLFJDUOJSJA,B08FRMGWXQ,5.0,1609218633016,Video Games,eXtremeRate 2 Pack Black Border Transparent HD...,[],"[Video Games, Nintendo Switch, Accessories, Fa...",14.59
54967,AERQISDPMPFJZKZ7P6A5FGX6RP5Q,B0000AHOOK,5.0,1415342350000,Video Games,WWE SmackDown! Here Comes the Pain - PlayStati...,"[Product Description, TV-style presentation ha...","[Video Games, Legacy Systems, PlayStation Syst...",None
54968,AEFPHMM7CLX4UJNXJFQF4ZF5GNAA,B07P27XFP7,5.0,1599585146628,Video Games,Shin Megami Tensei: Strange Journey Redux - Ni...,"[""A spatial distortion full of demons appeared...","[Video Games, Legacy Systems, Nintendo Systems...",69.95


In [11]:
# Define preprocessing steps for each column
preprocessing_pipeline = ColumnTransformer(
    transformers=tfm,
    remainder='drop'  # Drop any columns not specified in transformers
)

# Create a pipeline object
item_metadata_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline)
])

# Fit the pipeline
item_metadata_pipeline.fit(fit_df)

# Transform the data (useful for training)
transformed_item_metadata = item_metadata_pipeline.transform(train_full_df)

# For demonstration, print the shape of the transformed features and a few rows
print(f"Transformed Item Metadata Shape: {transformed_item_metadata.shape}")

Transformed Item Metadata Shape: (54970, 2159)


# Persist

In [12]:
import joblib
import pickle

In [13]:
train_full_df.to_parquet("../data/train_item_features.parquet")
val_full_df.to_parquet("../data/val_item_features.parquet")

In [14]:
joblib.dump(item_metadata_pipeline, '../data/item_metadata_pipeline.joblib')

['../data/item_metadata_pipeline.joblib']